this notebook computes the measurements for the bulk study. The notebook is self contain, it only requieres some standar python libraries. 

Idea and implementation by Diego Guarin

In [216]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline

In [54]:
def get_info_from_txt(file):
    #function to read the landmark, iris and bounding box location from txt files
    shape=np.zeros((68,2),dtype=int)
    left_pupil = np.zeros((1,3),dtype=int)
    right_pupil = np.zeros((1,3),dtype=int)
    bounding_box = np.zeros((1,4),dtype=int)
    
    cont_landmarks = 0
    get_landmarks = 0
    
    get_leftpupil = 0
    cont_leftpupil = 0
    
    get_rightpupil = 0
    cont_rightpupil = 0
    
    get_boundingbox = 0
    cont_boundingbox = 0 
    with open(file, 'r') as file:
        for i,line in enumerate(file):    
            if i == 4:    
                get_landmarks=1
            if i == 72:
                get_landmarks=0
                
            if get_landmarks == 1:
                temp=(line[:]).split(',')
                shape[cont_landmarks,0]=int(temp[0])
                shape[cont_landmarks,1]=int(temp[1])
                cont_landmarks += 1
                
            if i == 74:
                get_leftpupil = 1
            if i == 77:
                get_leftpupil = 0
                
            if get_leftpupil == 1:
                left_pupil[0,cont_leftpupil]=int(line[:])
                cont_leftpupil += 1
    
            if i == 79:
                get_rightpupil = 1
            if i == 82:
                get_rightpupil = 0
                
            if get_rightpupil == 1:            
                right_pupil[0,cont_rightpupil]=int(line[:])
                cont_rightpupil += 1 
                
            if i == 84:
                get_boundingbox = 1
            if i == 88:
                get_boundingbox = 0
                 
            if get_boundingbox == 1 :
                bounding_box[0,cont_boundingbox] = int(line[:])
                cont_boundingbox +=1
     
    lefteye=[left_pupil[0,0],left_pupil[0,1],left_pupil[0,2]]
    righteye=[right_pupil[0,0],right_pupil[0,1],right_pupil[0,2]]
    
    if cont_boundingbox > 0 :
        boundingbox = [bounding_box[0,0],bounding_box[0,1],bounding_box[0,2],bounding_box[0,3]]    
    else:
        boundingbox = [0,0,0,0]
    
    if lefteye[2] < 0 and righteye[2] >0:
        #left eye is closed, use right iris diameter and locate iris in center of eye
        lefteye[2] = righteye[2]           
        x_eye = shape[42:47,0]
        y_eye = shape[42:47,1]
        lefteye[0] = int(np.round(np.mean(x_eye),0))
        lefteye[1] = int(np.round(np.mean(y_eye),0))
        #message = 'Left Eye Closed'
         
    if righteye[2] < 0 and lefteye[2] >0:
        #Right eye is closed, use left iris diameter and locate iris in center of eye
        righteye[2] = lefteye[2]
        x_eye = shape[36:41,0]
        y_eye = shape[36:41,1]
        righteye[0] = int(np.round(np.mean(x_eye),0))
        righteye[1] = int(np.round(np.mean(y_eye),0))
        #message = 'Right Eye Closed'
            
    if righteye[2] < 0 and lefteye[2] <0:
        #both eyes are closed, use 1/4 of eye lenght as radius
        #and the center of mass of the eye as the center
        
        #for left eye 
        x_eye = shape[42:47,0]
        y_eye = shape[42:47,1]
        lefteye[0] = int(np.round(np.mean(x_eye),0))
        lefteye[1] = int(np.round(np.mean(y_eye),0))
        lefteye[2] = int(np.round((shape[45,0]-lefteye[0])/2))
        
        #for right eye 
        x_eye = shape[36:41,0]
        y_eye = shape[36:41,1]
        righteye[0] = int(np.round(np.mean(x_eye),0))
        righteye[1] = int(np.round(np.mean(y_eye),0))
        righteye[2] = int(np.round((shape[39,0]-righteye[0])/2))
        #message = 'Both Eyes Closed'
           
    return shape, lefteye, righteye, boundingbox

def estimate_lines(circle_left, circle_right):
    #function to estimate the line that connects the center of the eyes and a 
    #new, perpendicular line in the middle
    
    x_1=circle_right[0]
    y_1=circle_right[1]
    
    x_2=circle_left[0]
    y_2=circle_left[1]
    
    #find the point in the middle of the line
    x_m=((x_2-x_1)/2)+x_1
    m=(y_2-y_1)/(x_2-x_1)   
    y_m=(y_1+m*(x_m-x_1))
    
    x_m=int(round(x_m,0))
    y_m=int(round(y_m,0))
    angle=np.arctan(m)
    
    
    points=[x_m,y_m]
    
    return angle, points

In [222]:
def find_bulk_metrics(file, conversion_scale = 11.77, valid_image = True):



    shape, left_eye, right_eye, bounding_box = get_info_from_txt(file)
    angle, center = estimate_lines(left_eye, right_eye)

    scale = conversion_scale/(2*((left_eye[2]+right_eye[2])/2))


    rot_angle = angle+np.pi/2
    rot_matrix=np.array([[np.cos(rot_angle), np.sin(rot_angle)],
                      [-np.sin(rot_angle), np.cos(rot_angle)]])
    rot_matrix_inv=np.array([[np.cos(rot_angle), -np.sin(rot_angle)],
                  [np.sin(rot_angle), np.cos(rot_angle)]])



    x=shape[:,0]
    y=shape[:,1]
    rot_x,rot_y=rot_matrix.dot([x-center[0],y-center[1]])

    #left side of the face

    x=shape[12:17,0]
    y=shape[12:17,1]
    rot_x,rot_y=rot_matrix.dot([x-center[0],y-center[1]])


    spline = UnivariateSpline(rot_x[::-1],rot_y[::-1], s=1) 
    new_x_left = np.linspace(rot_x.min(), rot_x.max(),100)
    new_y_left = spline(new_x_left)
    idx = np.argmin(new_y_left)
    max_bulk_y_left = new_y_left[idx]
    max_bulk_x_left = new_x_left[idx]
    dist_left = abs(scale*max_bulk_y_left)

    #Right side of the face

    x=shape[0:5,0]
    y=shape[0:5,1]
    rot_x,rot_y=rot_matrix.dot([x-center[0],y-center[1]])


    spline = UnivariateSpline(rot_x,rot_y, s=1) 
    new_x_right = np.linspace(rot_x.min(), rot_x.max(),100)
    new_y_right = spline(new_x_right)
    idx = np.argmax(new_y_right)
    max_bulk_y_right = new_y_right[idx]
    max_bulk_x_right = new_x_right[idx]
    dist_right = abs(scale*max_bulk_y_right)


    if valid_image: 
        straight_new_x_left, straight_new_y_left = rot_matrix_inv.dot([new_x_left, new_y_left])
        straight_new_x_left, straight_new_y_left = straight_new_x_left+center[0], straight_new_y_left+center[1]

        straight_new_x_right, straight_new_y_right = rot_matrix_inv.dot([new_x_right, new_y_right])
        straight_new_x_right, straight_new_y_right = straight_new_x_right+center[0], straight_new_y_right+center[1]

        point_border_x_left, point_border_y_left = rot_matrix_inv.dot([max_bulk_x_left, max_bulk_y_left])
        point_border_x_left, point_border_y_left = point_border_x_left + center[0], point_border_y_left + center[1]
        point_center_x_left, point_center_y_left = rot_matrix_inv.dot([max_bulk_x_left,0])
        point_center_x_left, point_center_y_left = point_center_x_left + center[0], point_center_y_left + center[1]

        point_border_x_right, point_border_y_right = rot_matrix_inv.dot([max_bulk_x_right, max_bulk_y_right])
        point_border_x_right, point_border_y_right = point_border_x_right + center[0], point_border_y_right + center[1]
        point_center_x_right, point_center_y_right = rot_matrix_inv.dot([max_bulk_x_right,0])
        point_center_x_right, point_center_y_right = point_center_x_right + center[0], point_center_y_right + center[1]


        center_line_x, center_line_y = rot_matrix_inv.dot([300,0])
        center_line_x, center_line_y = center_line_x + center[0], center_line_y+center[1]

        fig  = plt.figure(figsize=(9,10))
        ax = fig.add_subplot(1, 1, 1)
        plt.scatter(shape[:,0], -shape[:,1])
        plt.plot(center[0],-center[1],'r',marker='o',markersize=10)
        plt.plot(np.array([center[0],center_line_x]), -np.array([center[1],center_line_y]), linewidth=1, color='tab:red')
        plt.plot(np.array([left_eye[0], right_eye[0]]), -np.array([left_eye[1], right_eye[1]]), linewidth=1, color='tab:red')
        plt.plot(straight_new_x_left, -straight_new_y_left, color='k')
        plt.plot(straight_new_x_right, -straight_new_y_right, color = 'k')
        plt.plot(np.array([point_border_x_left]), -np.array([point_border_y_left]), marker='o', markersize=5, color = 'tab:green')
        plt.plot(np.array([point_center_x_left, point_border_x_left]), -np.array([point_center_y_left, point_border_y_left]), color = 'tab:green')
        plt.text((point_border_x_left-point_center_x_left)/2 + point_center_x_left, -(point_center_y_left-20),str(np.round(dist_left,2)) + ' cm', fontsize=12)
        plt.plot(np.array([point_border_x_right]), -np.array([point_border_y_right]), marker='o', markersize=5, color = 'tab:orange')
        plt.plot(np.array([point_center_x_right, point_border_x_right]), -np.array([point_center_y_right, point_border_y_right]), color = 'tab:orange')
        plt.text((point_center_x_right-point_border_x_right)/2 + point_border_x_right, -(point_center_y_right-20),str(np.round(dist_right,2)) + ' cm', fontsize=12)

        plt.plot(left_eye[0], -left_eye[1], marker='o', markersize=left_eye[2], markerfacecolor='none', markeredgecolor= 'tab:red')
        plt.plot(right_eye[0], -right_eye[1], marker='o', markersize=right_eye[2], markerfacecolor='none', markeredgecolor= 'tab:red')

        frame1 = plt.gca()
        frame1.axes.xaxis.set_ticklabels([])
        frame1.axes.yaxis.set_ticklabels([])
        plt.savefig(file[:-3]+'png')
        plt.close()
        
    return dist_left, dist_right

In [227]:
folder = r'C:\Users\guarind\Documents\GitHub\Bulk_meatrics\data\text_files'
files = [f for f in os.listdir(folder) if f.endswith('.txt')]

data = pd.DataFrame(columns=['id', 'left (cm)', 'right (cm)'])
cl0=[]
cl1=[]
cl2=[]
for file in files:
    left, right = find_bulk_metrics(os.path.join(folder, file))
    cl0.append(file[:-4])
    cl1.append(left)
    cl2.append(right)
    
data['id'] = cl0
data['left (cm)'] = cl1
data['right (cm)'] = cl2
    
data.to_csv(os.path.join(folder, 'results.csv'))


C:\Users\guarind\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [225]:
data

,id,left (cm),right (cm)
0,7071738_post_repose,79.361004,77.607819
1,7071738_post_smile,86.098064,82.327255
2,7071738_pre_repose,69.347869,70.632221
3,7071738_pre_smile,75.101425,77.266804
4,7094090_post_repose,61.932743,58.991390
5,7094090_post_smile,66.469613,63.131542
6,7094090_pre_repose,79.239601,64.470981
7,7094090_pre_smile,67.913772,63.278301
8,7373065_post_repose,76.649366,74.335159
9,7373065_post_smile,59.392412,57.083576
